In [5]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [6]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [7]:
import pandas as pd

books = pd.read_csv('..\\data\\cleaned\\books_cleaned.csv')

In [8]:
books

isbn13      isbn10  \
0     9780002005883  0002005883   
1     9780002261982  0002261987   
2     9780006178736  0006178731   
3     9780006280897  0006280897   
4     9780006280934  0006280935   
...             ...         ...   
5192  9788172235222  8172235224   
5193  9788173031014  8173031010   
5194  9788179921623  817992162X   
5195  9788185300535  8185300534   
5196  9789027712059  9027712050   

                                                  title  \
0                                                Gilead   
1                                          Spider's Web   
2                                        Rage of angels   
3                                        The Four Loves   
4                                   The Problem of Pain   
...                                                 ...   
5192                                  Mistaken Identity   
5193                                Journey to the East   
5194  The Monk Who Sold His Ferrari: A Fable About F...   
5195                                          I Am that   
5196                           The Berlin Phenomenology   

                                           authors  \
0                               Marilynne Robinson   
1                  Charles Osborne;Agatha Christie   
2                                   Sidney Sheldon   
3                              Clive Staples Lewis   
4                              Clive Staples Lewis   
...                                            ...   
5192                              Nayantara Sahgal   
5193                                 Hermann Hesse   
5194                                  Robin Sharma   
5195  Sri Nisargadatta Maharaj;Sudhakar S. Dikshit   
5196                 Georg Wilhelm Friedrich Hegel   

                         categories  \
0                           Fiction   
1     Detective and mystery stories   
2                           Fiction   
3                    Christian life   
4                    Christian life   
...                             ...   
5192        Indic fiction (English)   
5193              Adventure stories   
5194               Health & Fitness   
5195                     Philosophy   
5196                        History   

                                              thumbnail  \
0     http://books.google.com/books/content?id=KQZCP...   
1     http://books.google.com/books/content?id=gA5GP...   
2     http://books.google.com/books/content?id=FKo2T...   
3     http://books.google.com/books/content?id=XhQ5X...   
4     http://books.google.com/books/content?id=Kk-uV...   
...                                                 ...   
5192  http://books.google.com/books/content?id=q-tKP...   
5193  http://books.google.com/books/content?id=rq6JP...   
5194  http://books.google.com/books/content?id=c_7mf...   
5195  http://books.google.com/books/content?id=Fv_JP...   
5196  http://books.google.com/books/content?id=Vy7Sk...   

                                            description  published_year  \
0     A NOVEL THAT READERS and critics have been eag...          2004.0   
1     A new 'Christie for Christmas' -- a full-lengt...          2000.0   
2     A memorable, mesmerizing heroine Jennifer -- b...          1993.0   
3     Lewis' work on the nature of love divides love...          2002.0   
4     "In The Problem of Pain, C.S. Lewis, one of th...          2002.0   
...                                                 ...             ...   
5192  On A Train Journey Home To North India After L...          2003.0   
5193  This book tells the tale of a man who goes on ...          2002.0   
5194  Wisdom to Create a Life of Passion, Purpose, a...          2003.0   
5195  This collection of the timeless teachings of o...          1999.0   
5196  Since the three volume edition ofHegel's Philo...          1981.0   

      average_rating  num_pages  ratings_count  \
0               3.85      247.0          361.0   
1               3.83      241.0         5164.0   
2    

In [9]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [10]:
books["tagged_description"].to_csv("..\\data\\processed\\tagged_description.txt",
                                   sep = "\n",
                                   index = False,
                                   header = False)

In [11]:
from langchain.document_loaders import TextLoader

In [ ]:
raw_documents = TextLoader("..\\data\\processed\\tagged_description.txt", encoding="utf-8").load()

In [13]:
from langchain.text_splitter import CharacterTextSplitter

In [14]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1168, which is longer than the specified 500
Created a chunk of size 1214, which is longer than the specified 500
Created a chunk of size 960, which is longer than the specified 500
Created a chunk of size 843, which is longer than the specified 500
Created a chunk of size 881, which is longer than the specified 500
Created a chunk of size 1088, which is longer than the specified 500
Created a chunk of size 1189, which is longer than the specified 500
Created a chunk of size 513, which is longer than the specified 500
Created a chunk of size 752, which is longer than the specified 500
Created a chunk of size 728, which is longer than the specified 500
Created a chunk of size 721, which is longer than the specified 500
Created a chunk of size 1267, which is longer than the specified 500
Created a chunk of size 681, which is longer than the specified 500
Created a chunk of size 553, which is longer than the specified 500
Created a chunk of size 521, which is longe

In [15]:
model_id = "sentence-transformers/all-MiniLM-L6-v2"
# Get the token from environment variables
huggingface_token = os.getenv("HF_TOKEN")

In [16]:
documents[0]

Document(metadata={'source': '..\\data\\processed\\tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest 

In [17]:
embeddings = HuggingFaceEmbeddings(
    model_name=model_id
)


C:\Users\mosta\AppData\Local\Temp\ipykernel_29832\1006448202.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [18]:
# Create the vector store with your documents
db_books = Chroma.from_documents(
    documents,
    embedding=embeddings)

In [19]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(metadata={'source': '..\\data\\processed\\tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(metadata={'source': '..\\data\\processed\\tagged_description.txt'}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(metadata={'source': '..\\data\\processed\\tagged_description.txt'}, page_content="9780786808397 Introduce yo

In [20]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

isbn13      isbn10                                title  \
3747  9780786808069  0786808063  Baby Einstein: Neighborhood Animals   

                                authors        categories  \
3747  Marilyn Singer;Julie Aigner-Clark  Juvenile Fiction   

                                              thumbnail  \
3747  http://books.google.com/books/content?id=X9a4P...   

                                            description  published_year  \
3747  Children will discover the exciting world of t...          2001.0   

      average_rating  num_pages  ratings_count  \
3747            3.89       16.0          180.0   

                       title_and_subtitle  \
3747  Baby Einstein: Neighborhood Animals   

                                     tagged_description  
3747  9780786808069 Children will discover the excit...

In [21]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    """
    Retrieves book recommendations based on semantic similarity to the input query.
    
    This function performs a semantic similarity search against a database of books
    and returns matching books as a pandas DataFrame.
    
    Args:
        query (str): The search query text to find semantically similar books.
        top_k (int, optional): Maximum number of recommendations to return. Defaults to 10.
            Note: Currently retrieves 50 results internally regardless of this parameter.
    
    Returns:
        pd.DataFrame: A filtered DataFrame containing books that match the query,
            with all columns from the original books DataFrame preserved.
    
    Example:
        >>> sci_fi_books = retrieve_semantic_recommendations("space adventure with aliens", top_k=5)
    """
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [22]:
retrieve_semantic_recommendations("A book to teach children about nature")

isbn13      isbn10  \
31    9780007105045  0007105045   
59    9780007151240  0007151241   
267   9780060882600  0060882603   
324   9780060959036  0060959037   
383   9780061144899  0061144894   
404   9780064402453  0064402452   
406   9780064403870  0064403874   
407   9780064404419  0064404412   
416   9780064406925  006440692X   
427   9780064434874  0064434877   
429   9780064434980  0064434982   
436   9780064471961  0064471969   
442   9780067575208  006757520X   
568   9780140139976  0140139974   
731   9780141015088  014101508X   
991   9780192862099  019286209X   
1077  9780240806082  0240806085   
1639  9780374422080  0374422087   
1642  9780374522599  0374522596   
1880  9780385732550  0385732554   
1907  9780393315110  0393315118   
1943  9780393959055  0393959058   
2126  9780439569729  0439569729   
2164  9780439994798  0439994799   
2409  9780451456816  0451456815   
2410  9780451457592  0451457595   
2457  9780465016907  0465016901   
2497  9780486256429  0486256421   
2718  9780553382617  0553382616   
2776  9780553585803  0553585800   
2805  9780571084838  0571084834   
2827  9780571207565  0571207561   
2897  9780618111206  0618111204   
3000  9780671631987  0671631985   
3577  9780762413935  076241393X   
3747  9780786808069  0786808063   
3748  9780786808373  0786808373   
3749  9780786808380  0786808381   
3750  9780786808397  078680839X   
3765  9780786819119  0786819111   
3834  9780802431486  0802431488   
4185  9780878937202  087893720X   
4264  9780941807555  094180755X   
4331  9781400033850  1400033853   
4541  9781421509594  1421509598   
4560  9781551114125  1551114127   
4606  9781561709335  1561709336   
4655  9781567921571  1567921574   
4713  9781576839546  1576839540   
4898  9781593851170  1593851170   

                                                  title  \
31                                        Tree and Leaf   
59                                       The Family Way   
267                       The Annotated Charlotte's Web   
324                                     Prodigal Summer   
383                                When the Heart Waits   
404                          Racso and the Rats of NIMH   
406                 R-T, Margaret, and the Rats of NIMH   
407                                  The Rainbow People   
416                                  Winter on the Farm   
427                          Christmas in the Big Woods   
429                                The Deer in the Wood   
436                             Shade's Children (rack)   
442                                 The Sense of Wonder   
568                                         Sailor Song   
731                           How to Breathe Underwater   
991                                 The Origins of Life   
1077                          Directing the Documentary   
1639                             Everything on a Waffle   
1642                              The Control of Nature   
1880                                          The Giver   
1907  Uncommon Ground: Rethinking the Human Place in...   
1943       Walden ; And, Resistance to Civil Government   
2126        SCHOLASTIC SUCCESS WITH 4TH GRADE(WORKBOOK)   
2164                                 His Dark Materials   
2409                                  Not of Woman Born   
2410                                   The River's Gift   
2457                      The Drama of the Gifted Child   
2497                                          Galapagos   
2718                              The Shelters of Stone   
2776                                Forty Signs of Rain   
2805                                  Lord of the Flies   
2827                       Collected Children's Stories   
2897                                        Once a Wolf   
3000       Teach Your Child to Read in 100 Easy Lessons   
3577    The Classic Treasury of Hans Christian Andersen   
3747                Baby Einstein: Neighborhood Animals   
3748                               Baby Eins